In [37]:
from src.controller.HwpController import HwpController

import re, uuid
import numpy as np

In [73]:
controller = HwpController()
path = '/Users/dobby/Desktop/Workspace/3월 계측보고서(신안산선 2공구 장하정거장).hwp'

controller.get_table_list(path, '일자별 계측 현황')
table_list = controller.get_list()


unmatched field end


In [74]:
title = ['진동속도(cm/s)', '진동레벨(dB(V))', '소음레벨(dB(A))']

columns = [ [ item['text'] for item in items if item['row'] == '0' or item['row'] == '1' ] for items in table_list ]
columns = set(tuple(items) for items in columns)
columns = [ item for items in columns for item in items ]

In [75]:
dict_list = [[item for item in items[len(columns):]]for items in table_list]

In [96]:
result_list = []

for items in dict_list:
    row_list = list(set([int(item['row']) for item in items]))
    for row in row_list:
        temp_list = [item for item in items if int(item['row']) == row]
        result_list.append(temp_list)

result_list

[[{'row': '2', 'col': '0', 'rowspan': '2', 'text': '3월1일'},
  {'row': '2', 'col': '1', 'rowspan': '1', 'text': '다인모듈'},
  {'row': '2', 'col': '2', 'rowspan': '1', 'text': '1회'},
  {'row': '2', 'col': '3', 'rowspan': '1', 'text': '7:00'},
  {'row': '2', 'col': '4', 'rowspan': '1', 'text': '0.2'},
  {'row': '2', 'col': '5', 'rowspan': '1', 'text': '75'},
  {'row': '2', 'col': '6', 'rowspan': '1', 'text': '75'},
  {'row': '2', 'col': '7', 'rowspan': '1', 'text': '0.053'},
  {'row': '2', 'col': '8', 'rowspan': '1', 'text': '56.46'},
  {'row': '2', 'col': '9', 'rowspan': '1', 'text': '53.23'}],
 [{'row': '3', 'col': '1', 'rowspan': '1', 'text': '다인모듈'},
  {'row': '3', 'col': '2', 'rowspan': '1', 'text': '2회'},
  {'row': '3', 'col': '3', 'rowspan': '1', 'text': '17:21'},
  {'row': '3', 'col': '4', 'rowspan': '1', 'text': '0.2'},
  {'row': '3', 'col': '5', 'rowspan': '1', 'text': '75'},
  {'row': '3', 'col': '6', 'rowspan': '1', 'text': '75'},
  {'row': '3', 'col': '7', 'rowspan': '1', 'text'

In [97]:
for idx, items in enumerate(result_list):
    head_data = [item for item in items if int(item['rowspan']) > 1 and int(item['col']) == 0]
    merge_data = [item for item in items if int(item['rowspan']) > 1 and int(item['col']) != 0]
    
    if head_data:
        real_head_data = head_data

    if merge_data:
        print(f'merge data: {merge_data}')
        real_data = merge_data

    print(f'head data: {head_data}')
    print(f'merge data: {merge_data}')
    print(f'real head data: {real_head_data}')
    print(f'real data: {real_data}')

    # if real_data and not any(data in items for data in real_data):
    #     for data in reversed(real_data):
    #         result_list[idx].insert(0, data)

    # if real_head_data and not any(data in items for data in real_head_data):
    #     for data in reversed(real_head_data):
    #         result_list[idx].insert(0, data)        
            
    print(result_list[idx])


head data: [{'row': '2', 'col': '0', 'rowspan': '2', 'text': '3월1일'}]
merge data: []
real head data: [{'row': '2', 'col': '0', 'rowspan': '2', 'text': '3월1일'}]
real data: [{'row': '14', 'col': '1', 'rowspan': '3', 'text': '1'}, {'row': '14', 'col': '2', 'rowspan': '3', 'text': '7:40'}]
[{'row': '2', 'col': '0', 'rowspan': '2', 'text': '3월1일'}, {'row': '2', 'col': '1', 'rowspan': '1', 'text': '다인모듈'}, {'row': '2', 'col': '2', 'rowspan': '1', 'text': '1회'}, {'row': '2', 'col': '3', 'rowspan': '1', 'text': '7:00'}, {'row': '2', 'col': '4', 'rowspan': '1', 'text': '0.2'}, {'row': '2', 'col': '5', 'rowspan': '1', 'text': '75'}, {'row': '2', 'col': '6', 'rowspan': '1', 'text': '75'}, {'row': '2', 'col': '7', 'rowspan': '1', 'text': '0.053'}, {'row': '2', 'col': '8', 'rowspan': '1', 'text': '56.46'}, {'row': '2', 'col': '9', 'rowspan': '1', 'text': '53.23'}]
head data: []
merge data: []
real head data: [{'row': '2', 'col': '0', 'rowspan': '2', 'text': '3월1일'}]
real data: [{'row': '14', 'col':